## Set-up

In [ ]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from PIL import Image, ImageDraw, ImageFont
import subprocess
import piexif
from datetime import datetime
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
import pickle 
from typing import List, Tuple, Any
import copy
import pandas as pd

# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
# MPS seems to crash every now and then
# elif torch.backends.mps.is_available():
#     device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

<h1>Config</h1>

In [42]:
# params stadspark
base_path='/Users/peter/playground/parkbridge'
src_image_path = f'{base_path}/source_images'
test_image_path = f'{base_path}/subset_test_images'

image_path = test_image_path
masks_path = f'{image_path}_masks'

homograpy_warped_images_folder = f'{base_path}/auto_warped_homography'
cropped_with_exif_images_folder = f'{base_path}/auto_warped_homography_cropped_with_exif'
translation_warped_images_folder = f'{base_path}/auto_warped_translation'
translation_and_rotation_warped_images_folder = f'{base_path}/auto_warped_translation_and_rotation'

# base_image_1 = 'IMG_20240820_074416.jpg'
# base_image_2 = 'IMG_20241120_123340.jpg'
# base_image_1 = 'IMG_20240528_080644.jpg'
# base_image_2 = 'IMG_20240528_080644_manual.jpg'

stop_motion_result_path=f'{base_path}/auto_parkbridge.mp4'

crop = True
crop_width = 3000
crop_height = 1800

matched_segments_bbox_expand = 0.05
min_segment_surface = 0.005
max_segment_surface = 0.075
match_distance_penalty_factor=0.075
match_distance_threshold=0.015



<h1>Functions</h1>

In [53]:
np.random.seed(3)
def show_anns(anns, borders=True):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:, :, 3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.5]])
        img[m] = color_mask 
        if borders:
            import cv2
            contours, _ = cv2.findContours(m.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
            # Try to smooth contours
            contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
            cv2.drawContours(img, contours, -1, (0, 0, 1, 0.4), thickness=1) 

    ax.imshow(img)

def downscale_image_by_percentage(image, scale_percent):
    """
    Downscale the image by a percentage while maintaining the aspect ratio.

    Args:
        image (PIL.Image or numpy.ndarray): The input image to downscale.
        scale_percent (float): The percentage to scale the image by (e.g., 50 for 50% of the original size).
    
    Returns:
        PIL.Image: The downscaled image.
    """
    # If the image is in NumPy format, convert it back to a PIL Image
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)
    
    # Calculate the new size based on the scale percentage
    width, height = image.size
    new_width = int(width * scale_percent / 100)
    new_height = int(height * scale_percent / 100)
    new_size = (new_width, new_height)
    
    # Resize the image to the new size
    downscaled_image = image.resize(new_size,  Image.Resampling.LANCZOS)
    
    return downscaled_image

def _load_image(path):
    image = Image.open(path)
    image = image.convert("RGB")
    image = downscale_image_by_percentage(image, scale_percent=100)
    image = np.array(image.convert("RGB"))
    return image

def plt_image(image):
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    
def expand_bounding_boxes_in_masks(image, masks_input, expand_percent=0.0):
    """
    Expand the bounding boxes of all masks by a given percentage, while ensuring they stay within image boundaries,
    and update the masks in place with the expanded bounding boxes.

    Args:
        masks (list): List of masks generated by SAM, each with a 'bbox' key.
        image (numpy.ndarray): The image from which the bounding box is extracted (to get dimensions).
        expand_percent (float): The percentage by which to expand the bounding boxes (e.g., 0.1 for 10%).

    Returns:
        list: The updated list of masks with modified 'bbox' values.
    """
    masks = copy.deepcopy(masks_input)

    # Get the dimensions of the image
    image_height, image_width = image.shape[:2]  # Height and width from the image dimensions
    
    # Iterate over all masks and expand their bounding boxes
    for mask in masks:
        x_min, y_min, width, height = mask['bbox']
        
        # Calculate the amount to expand in each direction
        expand_w = width * expand_percent
        expand_h = height * expand_percent
        
        # Calculate the new bounding box
        new_x_min = max(0, x_min - expand_w / 2)  # Ensure it doesn't go below 0
        new_y_min = max(0, y_min - expand_h / 2)  # Ensure it doesn't go below 0
        
        new_x_max = min(image_width, x_min + width + expand_w / 2)  # Ensure it doesn't exceed image width
        new_y_max = min(image_height, y_min + height + expand_h / 2)  # Ensure it doesn't exceed image height
        
        # Calculate new width and height based on the expanded coordinates
        new_width = new_x_max - new_x_min
        new_height = new_y_max - new_y_min
        
        # Update the 'bbox' in the mask with the expanded bounding box
        mask['bbox'] = (new_x_min, new_y_min, new_width, new_height)
    
    # Return the updated list of masks
    return masks

def find_matching_segment_with_distance_penalty(template, target_image, template_bbox, penalty_factor, distance_threshold):
    """
    Use cv2.matchTemplate to find the location of the segment in the target image,
    and penalize the match score based on how far the match is from the original template's bounding box.
    
    Args:
        template (numpy.ndarray): The extracted segment (template) from the first image.
        target_image (numpy.ndarray): The target image in which to search for the template.
        template_bbox (tuple): The bounding box of the template in the format (x_min, y_min, width, height).
        penalty_factor (float): A factor to control how much distance affects the score.
    
    Returns:
        tuple: Top-left corner of the best matching region in the target image, penalized match score.
    """
    # Convert both the template and target image to grayscale
    template_gray = cv2.cvtColor(template, cv2.COLOR_RGB2GRAY)
    target_image_gray = cv2.cvtColor(target_image, cv2.COLOR_RGB2GRAY)
    
    # Perform template matching using cv2.matchTemplate
    result = cv2.matchTemplate(target_image_gray, template_gray, cv2.TM_CCOEFF_NORMED)
    
    # Find the location with the highest match score
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    
    # max_loc is the top-left corner of the best match
    matched_top_left = max_loc
    
    # Extract the top-left corner of the original template's bounding box
    template_top_left = (template_bbox[0], template_bbox[1])
    
    # Calculate the Euclidean distance between the matched location and the template's original location
    distance = np.linalg.norm(np.array(matched_top_left) - np.array(template_top_left))
    
    # Apply a penalty to the match score based on the distance
    penalty = 1 / (1 + penalty_factor * distance)
    penalized_score = max_val * penalty
    if(penalized_score < distance_threshold):
        print(f'Score:{max_val} - Penalty score:{penalized_score}. Skipping')
        return None, None
    else:
        print(f'Score:{max_val} - Penalty score:{penalized_score}. Keeping')
        return matched_top_left, penalized_score

def process_all_masks(image, masks, target_image):
    """
    Process all masks, extract segments from the base image, and find corresponding matching regions in the target image.
    
    Args:
        image (numpy.ndarray): The input base image.
        masks (list): List of SAM-generated mask results (each containing 'segmentation').
        target_image (numpy.ndarray): The target image to search for matching regions.
    
    Returns:
        list: List of dictionaries with information about each match.
    """
    results = []
    
    # Loop over all the masks
    for idx, mask_data in enumerate(masks):
        # Extract the segmentation mask from the mask_data
        mask = mask_data['segmentation']
        bbox = mask_data['bbox']
        # Extract the segment from the base image using the mask
        extracted_segment = image[int(bbox[1]):int(bbox[1]+bbox[3]+1), int(bbox[0]):int(bbox[0]+bbox[2]+1)]
        # Find the corresponding matching region in the target image
        best_match_loc, match_score = find_matching_segment_with_distance_penalty(extracted_segment, target_image, bbox, penalty_factor=match_distance_penalty_factor, distance_threshold=match_distance_threshold)
        if(best_match_loc is None):
            continue
        
        # Store the result with necessary information
        results.append({
            'mask_index': idx,
            'best_match_loc': best_match_loc,
            'match_score': match_score,
            'segment_shape': extracted_segment.shape[:2]  # Height, width of the segment
        })
    
    return results

def plot_matches_side_by_side(base_image_name, target_image_name, base_image, target_image, match_results, masks):
    """
    Plot the original base image with segments on the left and the matched segments on the target image on the right.

    Args:
        base_image (numpy.ndarray): The original base image from which segments were extracted.
        target_image (numpy.ndarray): The target image where matches were found.
        match_results (list): List of dictionaries containing match information for each mask.
        masks (list): List of SAM-generated masks (with 'segmentation' and 'bbox').
    """
    # Create a copy of both images for displaying
    base_image_copy = base_image.copy()
    target_image_copy = target_image.copy()
    
    # Create a matplotlib figure with two subplots (side by side)
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Plot the base image with segment outlines on the left
    axes[0].imshow(base_image_copy)
    axes[0].set_title(f"Original {base_image_name} with Segments")
    
    # Plot the target image with match rectangles on the right
    axes[1].imshow(target_image_copy)
    axes[1].set_title(f"Matched Segments on {target_image_name}")
    
    # Loop through the match results and draw bounding boxes for both images
    for idx, (result, mask_data) in enumerate(zip(match_results, masks)):
        # Extract the original mask and bounding box (for the base image)
        mask_bbox = mask_data['bbox']
        x_min, y_min, width, height = mask_bbox
        
        # Draw rectangle and index in the center of the segment in the base image
        rect_base = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='blue', facecolor='none')
        axes[0].add_patch(rect_base)
        
        # Calculate center of the bounding box
        center_x_base = x_min + width / 2
        center_y_base = y_min + height / 2
        
        # Add index in the center of the base image's bounding box
        axes[0].text(center_x_base, center_y_base, str(idx), color='white', fontsize=12, ha='center', va='center')

        # Draw rectangles around the best match location on the target image
        top_left = result['best_match_loc']
        h, w = result['segment_shape']  # Height and width of the segment
        rect_target = patches.Rectangle(top_left, w, h, linewidth=2, edgecolor='green', facecolor='none')
        axes[1].add_patch(rect_target)
        
        # Calculate center of the bounding box on the target image
        center_x_target = top_left[0] + w / 2
        center_y_target = top_left[1] + h / 2
        
        # Add index in the center of the target image's bounding box
        axes[1].text(center_x_target, center_y_target, str(idx), color='white', fontsize=12, ha='center', va='center')
    
    # Hide axis ticks for both subplots
    axes[0].axis('off')
    axes[1].axis('off')
    
    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()
    
def plot_matches_and_warped_side_by_side(base_image_name, target_image_name, base_image, target_image, warped_image, match_results, masks):
    """
    Plot the original base image with segments on the left and the matched segments on the target image on the right.

    Args:
        base_image (numpy.ndarray): The original base image from which segments were extracted.
        target_image (numpy.ndarray): The target image where matches were found.
        match_results (list): List of dictionaries containing match information for each mask.
        masks (list): List of SAM-generated masks (with 'segmentation' and 'bbox').
    """
    # Create a copy of both images for displaying
    base_image_copy = base_image.copy()
    target_image_copy = target_image.copy()
    if(warped_image is not None):
        warped_image_copy = warped_image.copy()
    else:
        warped_image_copy = None
    # Create a matplotlib figure with three subplots (side by side)
    fig, axes = plt.subplots(1, 3, figsize=(12, 5))
    
    # Plot the base image with segment outlines on the left
    axes[0].imshow(base_image_copy)
    axes[0].set_title(f"Original {base_image_name} with Segments", fontsize=9)
    
    # Plot the target image with match rectangles 
    axes[1].imshow(target_image_copy)
    axes[1].set_title(f"Matched Segments on {target_image_name}", fontsize=9)
    
    # Plot the warped image on the right
    if(warped_image_copy is not None):
        axes[2].imshow(warped_image_copy)
        axes[2].set_title(f"Warped {target_image_name}", fontsize=9)
    
    # Loop through the match results and draw bounding boxes for base and target image
    template_bboxes = [masks[result['mask_index']]['bbox'] for result in match_results]
    target_bboxes = [(result['best_match_loc'][0], result['best_match_loc'][1], result['segment_shape'][1], result['segment_shape'][0]) for result in match_results]

    # for idx, (result, mask_data) in enumerate(zip(match_results, masks)):
    for idx, (result, mask_data) in enumerate(zip(match_results, template_bboxes)):
        print(f"{idx} - mask bbox:{mask_data} - best_match_loc:{result['best_match_loc']} ")
        # Extract the original mask and bounding box (for the base image)
        # mask_bbox = mask_data['bbox']
        mask_bbox = mask_data
        x_min, y_min, width, height = mask_bbox
        
        # Draw rectangle and index in the center of the segment in the base image
        rect_base = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='blue', facecolor='none')
        axes[0].add_patch(rect_base)
        
        # Calculate center of the bounding box
        center_x_base = x_min + width / 2
        center_y_base = y_min + height / 2
        
        # Add index in the center of the base image's bounding box
        axes[0].text(center_x_base, center_y_base, str(idx), color='white', fontsize=12, ha='center', va='center')

        # Draw rectangles around the best match location on the target image
        top_left = result['best_match_loc']
        h, w = result['segment_shape']  # Height and width of the segment
        rect_target = patches.Rectangle(top_left, w, h, linewidth=2, edgecolor='green', facecolor='none')
        axes[1].add_patch(rect_target)
        
        # Calculate center of the bounding box on the target image
        center_x_target = top_left[0] + w / 2
        center_y_target = top_left[1] + h / 2
        
        # Add index in the center of the target image's bounding box
        axes[1].text(center_x_target, center_y_target, str(idx), color='white', fontsize=12, ha='center', va='center')
    
    # Hide axis ticks
    axes[0].axis('off')
    axes[1].axis('off')
    axes[2].axis('off')
    
    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()    
            
def calculate_bounding_box_center(bbox):
    """
    Calculate the center of a bounding box.
    
    Args:
        bbox (tuple): Bounding box in the format (x_min, y_min, width, height).
    
    Returns:
        tuple: Center point (x, y) of the bounding box.
    """
    x_min, y_min, width, height = bbox
    center_x = x_min + width / 2
    center_y = y_min + height / 2
    return center_x, center_y

def calculate_homography_metrics(H):
    # Scale factors
    scale_x = np.sqrt(H[0, 0]**2 + H[0, 1]**2)
    scale_y = np.sqrt(H[1, 0]**2 + H[1, 1]**2)
    
    # Shear
    shear = (H[0, 0] * H[1, 0] + H[0, 1] * H[1, 1]) / (scale_x * scale_y)
    
    # Determinant
    determinant = H[0, 0] * H[1, 1] - H[0, 1] * H[1, 0]
    
    # Condition number
    u, s, vh = np.linalg.svd(H[:2, :2])  # Only consider the upper 2x2 for condition number
    condition_number = s[0] / s[-1] if s[-1] != 0 else np.inf
    
    # Perspective components
    perspective = np.linalg.norm(H[2, :2])
    
    # Angular deviation (rotation matrix approximation)
    rotation_angle = np.arctan2(H[1, 0], H[0, 0]) * (180 / np.pi)  # Convert to degrees
    
    return {
        "scale_x": scale_x,
        "scale_y": scale_y,
        "shear": shear,
        "determinant": determinant,
        "condition_number": condition_number,
        "perspective": perspective,
        "rotation_angle": rotation_angle
    }

def calculate_rigid_alignment(template_bboxes, target_bboxes):
    """
    Calculate the rigid transformation matrix (translation + rotation) to align
    the target image to the template.

    Args:
        template_bboxes (list of tuples): List of bounding boxes in the template image (x_min, y_min, width, height).
        target_bboxes (list of tuples): List of bounding boxes in the target image (x_min, y_min, width, height).

    Returns:
        numpy.ndarray: The 3x3 rigid transformation matrix.
    """
    def calculate_bounding_box_center(bbox):
        x, y, w, h = bbox
        return np.array([x + w / 2, y + h / 2])

    # Calculate the centers of the bounding boxes
    template_points = np.array([calculate_bounding_box_center(bbox) for bbox in template_bboxes])
    target_points = np.array([calculate_bounding_box_center(bbox) for bbox in target_bboxes])

    # Centralize the points
    template_center = np.mean(template_points, axis=0)
    target_center = np.mean(target_points, axis=0)
    template_centered = template_points - template_center
    target_centered = target_points - target_center

    # Singular Value Decomposition (SVD) to compute rotation
    U, _, Vt = np.linalg.svd(np.dot(target_centered.T, template_centered))
    R = np.dot(U, Vt)

    # Ensure no reflection in rotation
    if np.linalg.det(R) < 0:
        R[:, -1] *= -1

    # Compute translation
    t = template_center - np.dot(target_center, R)

    # Construct the rigid transformation matrix
    rigid_transform = np.eye(3)
    rigid_transform[:2, :2] = R
    rigid_transform[:2, 2] = t

    print(f'...rigid transform params: {calculate_homography_metrics(rigid_transform)}')

    return rigid_transform

def calculate_homography(template_bboxes, target_bboxes):
    """
    Calculate the homography matrix to warp the target image to match the template.
    
    Args:
        template_bboxes (list of tuples): List of bounding boxes in the template image (x_min, y_min, width, height).
        target_bboxes (list of tuples): List of bounding boxes in the target image (x_min, y_min, width, height).
    
    Returns:
        numpy.ndarray: The 3x3 homography matrix.
    """
    # Calculate the centers of the bounding boxes
    template_points = np.array([calculate_bounding_box_center(bbox) for bbox in template_bboxes])
    target_points = np.array([calculate_bounding_box_center(bbox) for bbox in target_bboxes])
    
    # Find the homography matrix using the points
    H, status = cv2.findHomography(target_points, template_points, cv2.RANSAC)
    print(f'...homography params: {calculate_homography_metrics(H)}')
    
    return H

def calculate_homography_with_rigid_transform_fallback(template_bboxes, target_bboxes):
    # Calculate the centers of the bounding boxes
    template_points = np.array([calculate_bounding_box_center(bbox) for bbox in template_bboxes])
    target_points = np.array([calculate_bounding_box_center(bbox) for bbox in target_bboxes])
    
    # Find the homography matrix using the points
    H, status = cv2.findHomography(target_points, template_points, cv2.RANSAC)
    metrics = calculate_homography_metrics(H)
    print(f'...homography params: {metrics}')
    if (metrics["condition_number"] > 10 or
        metrics["scale_x"] > 2 or metrics["scale_y"] > 2 or
        metrics["determinant"] < 0.1 or metrics["determinant"] > 2 or
        abs(metrics["rotation_angle"]) > 15):
            print("WARNING!!!! - distortion too high, falling back to rigid alignment.")
            rah = calculate_rigid_alignment(template_bboxes, target_bboxes)
            metrics = calculate_homography_metrics(H)
            print(f'...rigid transformation params: {rah}')
            return rah
    else:
        return H

def warp_target_image(target_image, homography_matrix, template_image_size):
    """
    Warp the target image using the homography matrix.
    
    Args:
        target_image (numpy.ndarray): The target image to be warped.
        homography_matrix (numpy.ndarray): The 3x3 homography matrix.
        template_image_size (tuple): The size of the template image (width, height).
    
    Returns:
        numpy.ndarray: The warped target image.
    """
    # Warp the target image to align with the template
    warped_image = cv2.warpPerspective(target_image, homography_matrix, template_image_size)
    
    return warped_image

def align_images_using_homography(base_image, target_image, match_results, masks):
    """
    Align the target image to the base image using homography based on matched bounding boxes.
    
    Args:
        base_image (numpy.ndarray): The base image (template).
        target_image (numpy.ndarray): The target image to be warped.
        match_results (list): List of dictionaries containing match information for each mask.
        masks (list): List of SAM-generated masks (with 'bbox' key).
    
    Returns:
        numpy.ndarray: The warped target image.
    """
    # Extract the bounding boxes from the masks and match results
    template_bboxes = [masks[result['mask_index']]['bbox'] for result in match_results]
    target_bboxes = [(result['best_match_loc'][0], result['best_match_loc'][1], result['segment_shape'][1], result['segment_shape'][0]) for result in match_results]
    print(f"Building homograph from {len(target_bboxes)} target_bboxes")
    print(f"...template:{template_bboxes}")
    print(f"...target:{target_bboxes}")
    # Calculate the homography matrix
    if(len(template_bboxes) < 4):
        print(f"Not enough matches to warp")
        return None
    H = calculate_homography(template_bboxes, target_bboxes)
    
    # Get the size of the base (template) image
    template_image_size = (base_image.shape[1], base_image.shape[0])  # (width, height)
    
    # Warp the target image to align with the base image
    warped_image = warp_target_image(target_image, H, template_image_size)
    
    return warped_image

def align_images_using_rigid_transform(base_image, target_image, match_results, masks):
    # Extract the bounding boxes from the masks and match results
    template_bboxes = [masks[result['mask_index']]['bbox'] for result in match_results]
    target_bboxes = [(result['best_match_loc'][0], result['best_match_loc'][1], result['segment_shape'][1], result['segment_shape'][0]) for result in match_results]
    print(f"Building homograph from {len(target_bboxes)} target_bboxes")
    print(f"...template:{template_bboxes}")
    print(f"...target:{target_bboxes}")
    H = calculate_rigid_alignment(template_bboxes, target_bboxes)
    
    # Get the size of the base (template) image
    template_image_size = (base_image.shape[1], base_image.shape[0])  # (width, height)
    
    # Warp the target image to align with the base image
    warped_image = warp_target_image(target_image, H, template_image_size)
    
    return warped_image

def align_images_using_homography_with_rigid_transform_fallback(base_image, target_image, match_results, masks):
    template_bboxes = [masks[result['mask_index']]['bbox'] for result in match_results]
    target_bboxes = [(result['best_match_loc'][0], result['best_match_loc'][1], result['segment_shape'][1], result['segment_shape'][0]) for result in match_results]
    print(f"Building homograph from {len(target_bboxes)} target_bboxes")
    print(f"...template:{template_bboxes}")
    print(f"...target:{target_bboxes}")

    if len(template_bboxes) < 4:
        print("Not enough matches to warp. Falling back to corners.")
        
        def get_bbox_corners_as_bboxes(bbox):
            x, y, w, h = bbox
            return [
                (x, y, 0, 0),          # Top-left as a bbox
                (x + w, y, 0, 0),      # Top-right as a bbox
                (x, y + h, 0, 0),      # Bottom-left as a bbox
                (x + w, y + h, 0, 0),  # Bottom-right as a bbox
            ]
        
        # Prepare fallback template and target bboxes
        template_points = []
        target_points = []
        for template_bbox, target_bbox in zip(template_bboxes, target_bboxes):
            template_points.extend(get_bbox_corners_as_bboxes(template_bbox))
            target_points.extend(get_bbox_corners_as_bboxes(target_bbox))
        
        print(f"Using corners as fallback: {len(template_points)} point pairs")
        if len(template_points) < 4:
            print("Still not enough points to compute homography, even with corners.")
            return None

        # Use the fallback points to calculate the homography
        H = calculate_homography_with_rigid_transform_fallback(template_points, target_points)
    else:
        H = calculate_homography_with_rigid_transform_fallback(template_bboxes, target_bboxes)

    template_image_size = (base_image.shape[1], base_image.shape[0])  # (width, height)    
    warped_image = warp_target_image(target_image, H, template_image_size)    
    return warped_image

def get_bbox_corners(bbox):
    """Returns the four corners of the bounding box."""
    x, y, w, h = bbox
    return [
        (x, y),               # Top-left
        (x + w, y),           # Top-right
        (x, y + h),           # Bottom-left
        (x + w, y + h)        # Bottom-right
    ]

def align_images_using_homography_corners(base_image, target_image, match_results, masks):
    """
    Align the target image to the base image using homography based on matched bounding boxes.
    
    Args:
        base_image (numpy.ndarray): The base image (template).
        target_image (numpy.ndarray): The target image to be warped.
        match_results (list): List of dictionaries containing match information for each mask.
        masks (list): List of SAM-generated masks (with 'bbox' key).
    
    Returns:
        numpy.ndarray: The warped target image.
    """
    # Extract the bounding boxes from the masks and match results
    template_bboxes = [masks[result['mask_index']]['bbox'] for result in match_results]
    target_bboxes = [(result['best_match_loc'][0], result['best_match_loc'][1], result['segment_shape'][1], result['segment_shape'][0]) for result in match_results]
    
    print(f"Building homograph from {len(target_bboxes)} target_bboxes")
    print(f"...template:{template_bboxes}")
    print(f"...target:{target_bboxes}")

    # Use the corners of the bounding boxes instead of just the center
    template_points = []
    target_points = []

    for template_bbox, target_bbox in zip(template_bboxes, target_bboxes):
        template_points.extend(get_bbox_corners(template_bbox))
        target_points.extend(get_bbox_corners(target_bbox))
        
    # Convert points to numpy arrays with the correct shape for cv2.findHomography
    template_points = np.array(template_points, dtype=np.float32)
    target_points = np.array(target_points, dtype=np.float32)        

    # Calculate the homography matrix
    if len(template_points) < 4:
        print(f"Not enough matches to warp")
        return None

    # H = calculate_homography(template_points, target_points)
    H, status = cv2.findHomography(target_points, template_points, cv2.RANSAC)

    
    # Get the size of the base (template) image
    template_image_size = (base_image.shape[1], base_image.shape[0])  # (width, height)
    
    # Warp the target image to align with the base image
    warped_image = warp_target_image(target_image, H, template_image_size)
    
    return warped_image

def mask_area_filter(image,masks,min_surf=min_segment_surface, max_surf=max_segment_surface):
    surface=image.shape[0]*image.shape[(1)]
    return [m for m in masks if ((m['area'] / surface) > min_surf and (m['area'] / surface) < max_surf)]

def match_segments(image,masks,target_image,min_segment_area=min_segment_surface,max_segment_area=max_segment_surface):
    filtered_masks = mask_area_filter(image, masks, min_segment_area, max_segment_area)
    results = process_all_masks(image, filtered_masks, target_image)
    return results,filtered_masks

def match_and_plot_segments(base_image_name,target_image_name, image,masks,target_image,min_segment_area=min_segment_surface,max_segment_area=max_segment_surface):
    filtered_masks = mask_area_filter(image, masks, min_segment_area, max_segment_area)
    results = process_all_masks(image, filtered_masks, target_image)
    plot_matches_side_by_side(base_image_name,target_image_name, image, target_image, results, filtered_masks)
    return results,filtered_masks

def match_warp_and_plot_segments_using_homography(base_image_name, target_image_name, base_image,masks,target_image,min_segment_area=min_segment_surface,max_segment_area=max_segment_surface):
    filtered_masks = mask_area_filter(base_image, masks, min_segment_area, max_segment_area)
    results = process_all_masks(base_image, filtered_masks, target_image)
    if(len(results) > 3):
        warped_image = align_images_using_homography(base_image, target_image, results, filtered_masks)
    else:
        print(f'Only {len(results)} matches. Using bboxes in stead of centers')
        warped_image = align_images_using_homography_corners(base_image, target_image, results, filtered_masks)
    if(warped_image is not None):
        plot_matches_and_warped_side_by_side(base_image_name, target_image_name, base_image, target_image, warped_image, results, filtered_masks)
    return results,filtered_masks,warped_image

def match_warp_and_plot_segments_using_homography_with_rigid_transform_fallback(base_image_name, target_image_name, base_image,masks,target_image,min_segment_area=min_segment_surface,max_segment_area=max_segment_surface):
    filtered_masks = mask_area_filter(base_image, masks, min_segment_area, max_segment_area)
    results = process_all_masks(base_image, filtered_masks, target_image)
    if(len(results) > 0):
        warped_image = align_images_using_homography_with_rigid_transform_fallback(base_image, target_image, results, filtered_masks)
    else:
        print(f'ERROR! No matches. Cannot align')
        return None,filtered_masks,None
    if(warped_image is not None):
        plot_matches_and_warped_side_by_side(base_image_name, target_image_name, base_image, target_image, warped_image, results, filtered_masks)
    else: 
        plot_matches_and_warped_side_by_side(base_image_name, target_image_name, base_image, target_image, None, results, filtered_masks)
    return results,filtered_masks,warped_image

def match_warp_and_plot_segments_using_rigid_transform(base_image_name, target_image_name, base_image,masks,target_image,min_segment_area=min_segment_surface,max_segment_area=max_segment_surface):
    filtered_masks = mask_area_filter(base_image, masks, min_segment_area, max_segment_area)
    results = process_all_masks(base_image, filtered_masks, target_image)
    if(len(results) > 3):
        warped_image = align_images_using_rigid_transform(base_image, target_image, results, filtered_masks)
    elif(len(results) > 0):
        print(f'Oops-Only {len(results)} matches. Use region corners')
        warped_image = align_images_using_homography_corners(base_image, target_image, results, filtered_masks)
        plot_matches_side_by_side(base_image_name, target_image_name, base_image, target_image, results, masks)
        return results,filtered_masks,warped_image
    else:
        print(f'ERROR {len(results)} matches. Cannot align')
        return None,filtered_masks,None
        
    if(warped_image is not None):
        plot_matches_and_warped_side_by_side(base_image_name, target_image_name, base_image, target_image, warped_image, results, filtered_masks)  
    return results,filtered_masks,warped_image

def create_stop_motion_movie(input_folder, output_file, frame_duration=2, transition_duration=1, fps=30):
    # Get all jpg files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.jpg')]
    image_files.sort()  # Sort the files to ensure correct order
    print(f"files {image_files}")

    # Get the dimensions of the first image
    first_image = cv2.imread(os.path.join(input_folder, image_files[0]))
    height, width = first_image.shape[:2]

    # Create a temporary video file
    temp_output = 'temp_output.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(temp_output, fourcc, fps, (width, height))

    for i in range(len(image_files)):
        current_img = cv2.imread(os.path.join(input_folder, image_files[i]))
        next_img = cv2.imread(os.path.join(input_folder, image_files[(i + 1) % len(image_files)]))
        
        # Hold the current image
        for _ in range(fps * frame_duration):
            out.write(current_img)
        
        # Cross-fade to the next image
        for j in range(int(fps * transition_duration)):
            alpha = j / (fps * transition_duration)
            blended = cv2.addWeighted(current_img, 1 - alpha, next_img, alpha, 0)
            out.write(blended)

    out.release()

    # Use FFmpeg to convert the temporary video to the final output with improved compression
    ffmpeg_cmd = [
        'ffmpeg',
        '-y',
        '-i', temp_output,
        '-c:v', 'libx264',
        '-preset', 'slow',
        '-crf', '23',
        '-vf', f'scale=-2:720',  # Scale to 720p, maintaining aspect ratio
        '-movflags', '+faststart',
        '-c:a', 'aac',
        '-b:a', '128k',
        output_file
    ]
    subprocess.run(ffmpeg_cmd, check=True)

    # Remove the temporary file
    os.remove(temp_output)

    print(f"Stop motion movie created: {output_file}")

def plot_overlayed(image,masks):
    filtered_masks = mask_area_filter(image,masks,0.0,1.0)
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    show_anns(filtered_masks)
    plt.axis('off')
    plt.show() 
    
def plot_overlayed_blank_nonmatched(image, masks):
    filtered_masks = mask_area_filter(image, masks, 0.0, 1.0)
    combined_mask = np.zeros(image.shape[:2], dtype=bool)
    for mask in filtered_masks:
        combined_mask = np.logical_or(combined_mask, mask["segmentation"])
    masked_image = np.zeros_like(image)
    masked_image[combined_mask] = image[combined_mask]
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].imshow(image)
    axes[0].set_title("Source Image")
    axes[0].axis('off')
    axes[1].imshow(masked_image)
    show_anns(filtered_masks)  # Assuming show_anns works with filtered_masks
    axes[1].set_title("Segmented Image")
    axes[1].axis('off')
    plt.tight_layout()
    plt.show()
    
def crop_center(image, crop_width, crop_height):
    """Crops the image from the center with given width and height."""
    img_width, img_height = image.size
    left = (img_width - crop_width) // 2
    top = (img_height - crop_height) // 2
    right = left + crop_width
    bottom = top + crop_height
    return image.crop((left, top, right, bottom))

def add_timestamp_from_exif(orig_image_path, image):
    # Timestamp from EXIF
    if(os.path.exists(orig_image_path)):
        orig_image = Image.open(orig_image_path)
        exif_data = piexif.load(orig_image.info.get('exif', b''))
        exif_datetime = exif_data.get('Exif', {}).get(piexif.ExifIFD.DateTimeOriginal)
    
    if os.path.exists(orig_image_path) and exif_datetime:
        dt = datetime.strptime(exif_datetime.decode('utf-8'), '%Y:%m:%d %H:%M:%S')
        timestamp_str = dt.strftime('%d-%b %H:%M')

        # Convert OpenCV BGR image to RGB for PIL
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(rgb_image)

        # Add timestamp using PIL
        draw = ImageDraw.Draw(pil_image)
        font_path = "/Library/Fonts/PTSans-Regular.ttf"
        font_size = 40
        font = ImageFont.truetype(font_path, font_size)
        img_width, img_height = pil_image.size
        text_position = (img_width // 2, 80)  # Top center, 12 pixels from the top
        draw.text(text_position, timestamp_str, font=font, fill=(255, 255, 255), anchor="ms")

        # Convert back to OpenCV BGR format
        result_image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
        # result_image = pil_image
        return result_image
    else:
        print('No EXIF datetime for the image')
        return image

def warp_target_image_with_translation(target_image, translation_vector, template_image_size):
    """
    Warp the target image using the translation vector.
    
    Args:
        target_image (numpy.ndarray): The target image to be warped.
        translation_vector (tuple): The (x, y) translation vector.
        template_image_size (tuple): The size of the template image (width, height).
    
    Returns:
        numpy.ndarray: The translated target image.
    """
    # Create a translation matrix
    translation_matrix = np.float32([[1, 0, translation_vector[0]], [0, 1, translation_vector[1]]])
    
    # Warp the target image using only translation
    translated_image = cv2.warpAffine(target_image, translation_matrix, template_image_size)
    
    return translated_image

def calculate_translation(template_bboxes, target_bboxes):
    """
    Calculate the average translation vector between template and target bounding boxes.
    
    Args:
        template_bboxes (list): List of bounding boxes from the template (base) image.
        target_bboxes (list): List of bounding boxes from the target image.
    
    Returns:
        tuple: The (x, y) translation vector.
    """
    # Initialize translation deltas
    delta_x = 0
    delta_y = 0
    
    # Calculate the translation for each corresponding pair of bounding boxes
    for template_bbox, target_bbox in zip(template_bboxes, target_bboxes):
        template_center_x = template_bbox[0] + template_bbox[2] / 2
        template_center_y = template_bbox[1] + template_bbox[3] / 2
        
        target_center_x = target_bbox[0] + target_bbox[2] / 2
        target_center_y = target_bbox[1] + target_bbox[3] / 2
        
        delta_x += template_center_x - target_center_x
        delta_y += template_center_y - target_center_y
    
    # Average the translation across all matched bounding boxes
    num_matches = len(template_bboxes)
    if num_matches == 0:
        return (0, 0)
    
    avg_delta_x = delta_x / num_matches
    avg_delta_y = delta_y / num_matches
    
    return (avg_delta_x, avg_delta_y)

def align_images_using_translation(base_image, target_image, match_results, masks):
    """
    Align the target image to the base image using translation based on matched bounding boxes.
    
    Args:
        base_image (numpy.ndarray): The base image (template).
        target_image (numpy.ndarray): The target image to be translated.
        match_results (list): List of dictionaries containing match information for each mask.
        masks (list): List of SAM-generated masks (with 'bbox' key).
    
    Returns:
        numpy.ndarray: The translated target image.
    """
    # Extract the bounding boxes from the masks and match results
    template_bboxes = [masks[result['mask_index']]['bbox'] for result in match_results]
    target_bboxes = [(result['best_match_loc'][0], result['best_match_loc'][1], result['segment_shape'][1], result['segment_shape'][0]) for result in match_results]
    print(f"Building translation from {len(target_bboxes)} target_bboxes")
    print(f"...template:{template_bboxes}")
    print(f"...target:{target_bboxes}")
    
    if len(template_bboxes) == 0 or len(target_bboxes) == 0:
        print("Not enough matches to perform translation")
        return None
    
    # Calculate the average translation vector
    translation_vector = calculate_translation(template_bboxes, target_bboxes)
    print(f"Calculated translation vector: {translation_vector}")
    
    # Get the size of the base (template) image
    template_image_size = (base_image.shape[1], base_image.shape[0])  # (width, height)
    
    # Warp (translate) the target image to align with the base image
    translated_image = warp_target_image_with_translation(target_image, translation_vector, template_image_size)
    
    return translated_image

def match_warp_and_plot_segments_using_translation(base_image_name, target_image_name, base_image,masks,target_image,min_segment_area=0.0015,max_segment_area=max_segment_surface):
    filtered_masks = mask_area_filter(base_image, masks, min_segment_area, max_segment_area)
    results = process_all_masks(base_image, filtered_masks, target_image)
    warped_image = None
    warped_image = align_images_using_translation(base_image, target_image, results, filtered_masks)
    if(warped_image is not None):
        plot_matches_and_warped_side_by_side(base_image_name, target_image_name, base_image, target_image, warped_image, results, filtered_masks)

    return results,filtered_masks,warped_image

def calculate_translation_and_rotation(template_bboxes, target_bboxes):
    """
    Calculate the average translation vector and rotation angle between template and target bounding boxes.
    
    Args:
        template_bboxes (list): List of bounding boxes from the template (base) image.
        target_bboxes (list): List of bounding boxes from the target image.
    
    Returns:
        tuple: The (x, y) translation vector and the average rotation angle (in degrees).
    """
    delta_x = 0
    delta_y = 0
    delta_angle = 0
    
    for template_bbox, target_bbox in zip(template_bboxes, target_bboxes):
        # Calculate the centers of the bounding boxes
        template_center_x = template_bbox[0] + template_bbox[2] / 2
        template_center_y = template_bbox[1] + template_bbox[3] / 2
        
        target_center_x = target_bbox[0] + target_bbox[2] / 2
        target_center_y = target_bbox[1] + target_bbox[3] / 2
        
        delta_x += template_center_x - target_center_x
        delta_y += template_center_y - target_center_y
        
        # Calculate the angle between the width/height ratios of the bounding boxes
        template_ratio = np.arctan2(template_bbox[3], template_bbox[2])
        target_ratio = np.arctan2(target_bbox[3], target_bbox[2])
        
        delta_angle += np.rad2deg(template_ratio - target_ratio)
    
    num_matches = len(template_bboxes)
    if num_matches == 0:
        return (0, 0, 0)
    
    avg_delta_x = delta_x / num_matches
    avg_delta_y = delta_y / num_matches
    avg_delta_angle = delta_angle / num_matches
    
    return (avg_delta_x, avg_delta_y, avg_delta_angle)

def align_images_using_translation_and_rotation(base_image, target_image, match_results, masks):
    """
    Align the target image to the base image using translation and rotation based on matched bounding boxes.
    
    Args:
        base_image (numpy.ndarray): The base image (template).
        target_image (numpy.ndarray): The target image to be transformed.
        match_results (list): List of dictionaries containing match information for each mask.
        masks (list): List of SAM-generated masks (with 'bbox' key).
    
    Returns:
        numpy.ndarray: The transformed target image.
    """
    template_bboxes = [masks[result['mask_index']]['bbox'] for result in match_results]
    target_bboxes = [(result['best_match_loc'][0], result['best_match_loc'][1], result['segment_shape'][1], result['segment_shape'][0]) for result in match_results]
    
    if len(template_bboxes) == 0 or len(target_bboxes) == 0:
        print("Not enough matches to perform transformation")
        return None
    
    # Calculate the average translation vector and rotation angle
    avg_delta_x, avg_delta_y, avg_delta_angle = calculate_translation_and_rotation(template_bboxes, target_bboxes)
    translation_vector = (avg_delta_x, avg_delta_y)
    
    print(f"Calculated translation vector: {translation_vector}, Rotation angle: {avg_delta_angle}")
    
    # Get the size of the base (template) image
    template_image_size = (base_image.shape[1], base_image.shape[0])  # (width, height)
    
    # Create a transformation matrix for both translation and rotation
    center = (template_image_size[0] // 2, template_image_size[1] // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, avg_delta_angle, 1.0)
    
    # Apply the translation to the transformation matrix
    rotation_matrix[0, 2] += translation_vector[0]
    rotation_matrix[1, 2] += translation_vector[1]
    
    # Warp (transform) the target image
    transformed_image = cv2.warpAffine(target_image, rotation_matrix, template_image_size)
    
    return transformed_image

def match_warp_and_plot_segments_using_translation_and_rotation(base_image_name, target_image_name, base_image,masks,target_image,min_segment_area=0.0015,max_segment_area=max_segment_surface):
    filtered_masks = mask_area_filter(base_image, masks, min_segment_area, max_segment_area)
    results = process_all_masks(base_image, filtered_masks, target_image)
    warped_image = None    
    if(len(results) > 3):
        warped_image = align_images_using_translation_and_rotation(base_image, target_image, results, filtered_masks)
        if(warped_image is not None):
            plot_matches_and_warped_side_by_side(base_image_name, target_image_name, base_image, target_image, warped_image, results, filtered_masks)
    return results,filtered_masks,warped_image

def get_or_generate_masks(mask_generator: Any, image_path: str, saved_masks_folder: str) -> List[dict]:
    """
    Retrieves saved masks if they exist; otherwise, generates masks using the provided generator,
    saves them, and returns the generated masks.

    Args:
        mask_generator (Any): The SAM2 mask generator instance.
        image_path (str): Path to the input image.
        saved_masks_folder (str): Path to the folder where masks are saved/loaded.

    Returns:
        List[dict]: List of masks for the image.
    """
    # Ensure the saved masks folder exists
    os.makedirs(saved_masks_folder, exist_ok=True)

    # Derive a unique filename for the mask based on the image path
    image_filename = os.path.basename(image_path)
    mask_filename = os.path.splitext(image_filename)[0] + "_masks.pkl"
    mask_filepath = os.path.join(saved_masks_folder, mask_filename)

    # Check if the mask file exists
    if os.path.exists(mask_filepath):
        # Load and return the saved masks
        with open(mask_filepath, 'rb') as file:
            masks = pickle.load(file)
        print(f"Loaded masks from {mask_filepath}")
        return masks

    # If masks are not found, generate them
    print(f"Masks not found. Generating new masks for {image_path}...")
    import cv2
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")

    # Generate masks using the provided mask generator
    masks = mask_generator.generate(image)

    # Save the generated masks
    with open(mask_filepath, 'wb') as file:
        pickle.dump(masks, file)
    print(f"Generated masks saved to {mask_filepath}")

    return masks

def load_image(image_path: str) -> Any:
    """Load an image from the given path."""
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def process_images_and_masks(
    folder_path: str, 
    saved_masks_folder: str, 
    mask_generator: Any
) -> Tuple[List[Any], List[List[dict]]]:
    """
    Processes all images in the specified folder, generates or retrieves masks for each,
    and returns a list of images and their corresponding masks.

    Args:
        folder_path (str): Path to the folder containing images.
        saved_masks_folder (str): Path to the folder where masks are saved/loaded.
        mask_generator (Any): The SAM2 mask generator instance.

    Returns:
        Tuple[List[Any], List[List[dict]]]: A tuple of lists containing images and their masks.
    """
    # Ensure the saved masks folder exists
    os.makedirs(saved_masks_folder, exist_ok=True)

    # Sort the list of image files in the folder
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.jpg')])

    # Load all images and generate/retrieve their masks
    images_list = []
    masks_list = []

    for file in image_files:
        image_path = os.path.join(folder_path, file)
        # Load image
        image = load_image(image_path)
        images_list.append(image)

        # Generate or retrieve masks
        masks = get_or_generate_masks(mask_generator, image_path, saved_masks_folder)
        masks_list.append(masks)

    return image_files,images_list, masks_list

def calculate_homography_and_display(image1, image2, masks1, masks2):
    """
    Calculates the homography based on IoU of SAM masks, applies the homography, 
    and displays the first image, second image, and the warped image side by side.

    Args:
        image1 (np.ndarray): First image (reference).
        image2 (np.ndarray): Second image to align.
        masks1 (List[dict]): Masks from SAM for the first image.
        masks2 (List[dict]): Masks from SAM for the second image.
    """
    def compute_iou(mask1, mask2):
        """Computes IoU between two binary masks."""
        intersection = np.logical_and(mask1, mask2).sum()
        union = np.logical_or(mask1, mask2).sum()
        return intersection / union

    # Match masks based on IoU
    matched_points = []
    for mask1 in masks1:
        seg1 = mask1["segmentation"]
        best_match = None
        best_iou = 0
        for mask2 in masks2:
            seg2 = mask2["segmentation"]
            iou = compute_iou(seg1, seg2)
            if iou > best_iou:
                best_iou = iou
                best_match = mask2
        
        if best_match and best_iou > 0.3:  # Keep only meaningful matches
            # Use bounding box centers as keypoints
            bbox1 = mask1["bbox"]
            bbox2 = best_match["bbox"]
            center1 = (bbox1[0] + bbox1[2] / 2, bbox1[1] + bbox1[3] / 2)
            center2 = (bbox2[0] + bbox2[2] / 2, bbox2[1] + bbox2[3] / 2)
            matched_points.append((center1, center2))

    # Check if we have enough points to calculate homography
    if len(matched_points) >= 4:
        # Separate matched points into source and destination
        src_points = np.array([p[0] for p in matched_points], dtype=np.float32)
        dst_points = np.array([p[1] for p in matched_points], dtype=np.float32)

        # Calculate the homography matrix
        H, status = cv2.findHomography(dst_points, src_points, cv2.RANSAC)

        # Warp the second image to align with the first
        warped_image = cv2.warpPerspective(image2, H, (image1.shape[1], image1.shape[0]))
        # Display the images side by side
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.title("First Image (Reference)")
        plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.title("Second Image (To Align)")
        plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.title("Warped Image")
        plt.imshow(cv2.cvtColor(warped_image, cv2.COLOR_BGR2RGB))
        plt.axis("off")

        plt.tight_layout()
        plt.show()
        
    else:
        print("Not enough matched points to calculate homography.")
        # Display the images side by side
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 2, 1)
        plt.title("First Image (Reference)")
        plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
        plt.axis("off")

        plt.subplot(1, 2, 2)
        plt.title("Second Image (To Align)")
        plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
        plt.axis("off")

        plt.tight_layout()
        plt.show()

def calculate_homography_with_translation(image1, image2, masks1, masks2):
    """
    Calculates the homography by first estimating translation of SAM masks
    and then refining using feature matching. Displays the results.

    Args:
        image1 (np.ndarray): First image (reference).
        image2 (np.ndarray): Second image to align.
        masks1 (List[dict]): Masks from SAM for the first image.
        masks2 (List[dict]): Masks from SAM for the second image.
    """
    def compute_iou(mask1, mask2):
        """Computes IoU between two binary masks."""
        intersection = np.logical_and(mask1, mask2).sum()
        union = np.logical_or(mask1, mask2).sum()
        return intersection / union

    # Step 1: Coarse Matching Based on IoU
    matched_masks = []
    for mask1 in masks1:
        seg1 = mask1["segmentation"]
        best_match = None
        best_iou = 0
        for mask2 in masks2:
            seg2 = mask2["segmentation"]
            iou = compute_iou(seg1, seg2)
            if iou > best_iou:
                best_iou = iou
                best_match = mask2
        
        if best_match and best_iou > 0.3:  # Lower IoU threshold
            matched_masks.append((mask1, best_match))

    if not matched_masks:
        raise ValueError("No sufficiently matching masks found between the images.")

    # Step 2: Estimate Rough Translation
    src_points = []
    dst_points = []
    for mask1, mask2 in matched_masks:
        bbox1 = mask1["bbox"]
        bbox2 = mask2["bbox"]

        # Calculate bounding box centers
        center1 = (bbox1[0] + bbox1[2] / 2, bbox1[1] + bbox1[3] / 2)
        center2 = (bbox2[0] + bbox2[2] / 2, bbox2[1] + bbox2[3] / 2)

        src_points.append(center1)
        dst_points.append(center2)

    src_points = np.array(src_points, dtype=np.float32)
    dst_points = np.array(dst_points, dtype=np.float32)

    # Compute rough translation matrix
    translation_matrix, _ = cv2.estimateAffinePartial2D(dst_points, src_points)
    translated_image2 = cv2.warpAffine(image2, translation_matrix, (image1.shape[1], image1.shape[0]))

    # Step 3: Feature Matching for Refinement
    orb = cv2.ORB_create(nfeatures=500)
    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(translated_image2, None)

    if descriptors1 is None or descriptors2 is None:
        raise ValueError("Feature descriptors could not be computed for one of the images.")

    # Match features
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)

    # Map matched keypoints to original image coordinates
    matched_src_points = np.float32([keypoints1[m.queryIdx].pt for m in matches])
    matched_dst_points = np.float32([keypoints2[m.trainIdx].pt for m in matches])

    # Compute final homography
    H, status = cv2.findHomography(matched_dst_points, matched_src_points, cv2.RANSAC)

    # Warp image2 to align with image1
    warped_image = cv2.warpPerspective(image2, H, (image1.shape[1], image1.shape[0]))

    # Step 4: Display Results
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title("First Image (Reference)")
    plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title("Second Image (To Align)")
    plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title("Warped Image")
    plt.imshow(cv2.cvtColor(warped_image, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.tight_layout()
    plt.show()

def calculate_shape_distance_based_matching(image1, image2, masks1, masks2, distance_weight=0.5):
    """
    Matches SAM segments between two images based on shape similarity and spatial proximity.
    Calculates and applies the homography.

    Args:
        image1 (np.ndarray): First image (reference).
        image2 (np.ndarray): Second image to align.
        masks1 (List[dict]): Masks from SAM for the first image.
        masks2 (List[dict]): Masks from SAM for the second image.
        distance_weight (float): Weight for distance penalty in the matching score.

    Returns:
        np.ndarray: The warped version of image2.
    """
    def compute_hu_similarity(seg1, seg2):
        """Compute shape similarity using Hu Moments."""
        moments1 = cv2.moments(seg1.astype(np.uint8))
        moments2 = cv2.moments(seg2.astype(np.uint8))
        hu1 = cv2.HuMoments(moments1).flatten()
        hu2 = cv2.HuMoments(moments2).flatten()
        # Log-transform for better comparison
        hu1 = -np.sign(hu1) * np.log10(np.abs(hu1) + 1e-8)
        hu2 = -np.sign(hu2) * np.log10(np.abs(hu2) + 1e-8)
        return 1.0 / (1.0 + np.linalg.norm(hu1 - hu2))  # Higher is better

    def compute_distance_penalty(center1, center2):
        """Compute penalty based on Euclidean distance."""
        distance = np.linalg.norm(np.array(center1) - np.array(center2))
        return 1.0 / (1.0 + distance)  # Higher is better

    # Match masks with combined shape and distance score
    matched_points = []
    for mask1 in masks1:
        seg1 = mask1["segmentation"]
        center1 = (
            mask1["bbox"][0] + mask1["bbox"][2] / 2,
            mask1["bbox"][1] + mask1["bbox"][3] / 2,
        )
        best_match = None
        best_score = 0
        for mask2 in masks2:
            seg2 = mask2["segmentation"]
            center2 = (
                mask2["bbox"][0] + mask2["bbox"][2] / 2,
                mask2["bbox"][1] + mask2["bbox"][3] / 2,
            )
            shape_similarity = compute_hu_similarity(seg1, seg2)
            distance_penalty = compute_distance_penalty(center1, center2)
            score = shape_similarity * (1 - distance_weight) + distance_penalty * distance_weight
            if score > best_score:
                best_score = score
                best_match = center2

        if best_match:
            matched_points.append((center1, best_match))

    if len(matched_points) < 4:
        raise ValueError("Not enough matched points to calculate homography.")

    # Compute homography
    src_points = np.array([p[0] for p in matched_points], dtype=np.float32)
    dst_points = np.array([p[1] for p in matched_points], dtype=np.float32)
    H, _ = cv2.findHomography(dst_points, src_points, cv2.RANSAC)

    # Warp the second image
    warped_image = cv2.warpPerspective(image2, H, (image1.shape[1], image1.shape[0]))

    # Display the images side by side
    import matplotlib.pyplot as plt
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title("First Image (Reference)")
    plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title("Second Image (To Align)")
    plt.imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title("Warped Image")
    plt.imshow(cv2.cvtColor(warped_image, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.tight_layout()
    plt.show()

    return warped_image

def calculate_and_display_best_matches(image1, image2, masks1, masks2, distance_weight=0.5, top_k=6):
    """
    Matches SAM segments between two images based on shape similarity and spatial proximity,
    calculates and applies the homography, and displays the best matches.

    Args:
        image1 (np.ndarray): First image (reference).
        image2 (np.ndarray): Second image to align.
        masks1 (List[dict]): Masks from SAM for the first image.
        masks2 (List[dict]): Masks from SAM for the second image.
        distance_weight (float): Weight for distance penalty in the matching score.
        top_k (int): Number of best matches to visualize.

    Returns:
        np.ndarray: The warped version of image2.
    """
    def compute_hu_similarity(seg1, seg2):
        """Compute shape similarity using Hu Moments."""
        moments1 = cv2.moments(seg1.astype(np.uint8))
        moments2 = cv2.moments(seg2.astype(np.uint8))
        hu1 = cv2.HuMoments(moments1).flatten()
        hu2 = cv2.HuMoments(moments2).flatten()
        hu1 = -np.sign(hu1) * np.log10(np.abs(hu1) + 1e-8)
        hu2 = -np.sign(hu2) * np.log10(np.abs(hu2) + 1e-8)
        return 1.0 / (1.0 + np.linalg.norm(hu1 - hu2))

    def compute_distance_penalty(center1, center2):
        """Compute penalty based on Euclidean distance."""
        distance = np.linalg.norm(np.array(center1) - np.array(center2))
        return 1.0 / (1.0 + distance)

    # Step 1: Match masks with combined shape and distance score
    match_scores = []
    for mask1 in masks1:
        seg1 = mask1["segmentation"]
        center1 = (
            mask1["bbox"][0] + mask1["bbox"][2] / 2,
            mask1["bbox"][1] + mask1["bbox"][3] / 2,
        )
        for mask2 in masks2:
            seg2 = mask2["segmentation"]
            center2 = (
                mask2["bbox"][0] + mask2["bbox"][2] / 2,
                mask2["bbox"][1] + mask2["bbox"][3] / 2,
            )
            shape_similarity = compute_hu_similarity(seg1, seg2)
            distance_penalty = compute_distance_penalty(center1, center2)
            score = shape_similarity * (1 - distance_weight) + distance_penalty * distance_weight
            match_scores.append((score, mask1, mask2, center1, center2))

    # Step 2: Select top-k matches
    match_scores = sorted(match_scores, key=lambda x: -x[0])[:top_k]
    matched_points = [(m[3], m[4]) for m in match_scores]

    if len(matched_points) < 4:
        raise ValueError("Not enough matched points to calculate homography.")

    # Step 3: Compute homography
    src_points = np.array([p[0] for p in matched_points], dtype=np.float32)
    dst_points = np.array([p[1] for p in matched_points], dtype=np.float32)
    H, _ = cv2.findHomography(dst_points, src_points, cv2.RANSAC)

    # Warp the second image
    warped_image = cv2.warpPerspective(image2, H, (image1.shape[1], image1.shape[0]))

    # Step 4: Visualize Matches
    image1_copy = image1.copy()
    image2_copy = image2.copy()
    image2_offset = image1.shape[1]  # Offset for second image in the combined image

    combined_image = np.concatenate((image1_copy, image2_copy), axis=1)

    plt.figure(figsize=(15, 5))
    plt.imshow(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))

    for i, (score, mask1, mask2, center1, center2) in enumerate(match_scores):
        # Draw segments
        color = tuple(np.random.randint(0, 255, size=(3,), dtype=int))  # Random color

        # Color segmentation in the first image
        seg1 = mask1["segmentation"]
        image1_copy[seg1] = color

        # Color segmentation in the second image
        seg2 = mask2["segmentation"]
        image2_copy[seg2] = color

        # Draw lines connecting matched points
        pt1_x, pt1_y = int(center1[0]), int(center1[1])
        pt2_x, pt2_y = int(center2[0] + image2_offset), int(center2[1])  # Adjust for offset

        plt.scatter(pt1_x, pt1_y, color='red', s=50, label="Image1 Match" if i == 0 else "")
        plt.scatter(pt2_x, pt2_y, color='blue', s=50, label="Image2 Match" if i == 0 else "")
        plt.plot([pt1_x, pt2_x], [pt1_y, pt2_y], color='yellow', linewidth=1)

    plt.axis("off")
    plt.legend()
    plt.show()

    return warped_image

def plot_images_with_masks(image_files,images,masks):
    base_image_ix = image_files.index(base_image)
    for idx,img in enumerate(images):
        print(idx,image_files[idx],len(masks[idx]))
        plot_overlayed_blank_nonmatched(img,masks[idx])

def process_and_display_homographies(images_list, masks_list):
    """
    Calculates homographies for a series of images relative to the first image,
    applies the transformations, and displays the results.

    Args:
        images_list (List[np.ndarray]): List of images to process.
        masks_list (List[List[dict]]): List of SAM masks corresponding to the images.
    """
    # Use the first image and its masks as the base
    base_image = images_list[0]
    base_masks = masks_list[0]

    # Iterate through the rest of the images
    for idx, (image, masks) in enumerate(zip(images_list[1:], masks_list[1:]), start=1):
        print(f"Processing image {idx + 1}/{len(images_list)}...")

        try:
            # Calculate homography and display the results
            # calculate_homography_and_display(base_image, image, base_masks, masks)
            # calculate_homography_with_translation(base_image, image, base_masks, masks)
            # calculate_shape_distance_based_matching(base_image, image, base_masks, masks)
            # calculate_and_display_matches_with_segments(base_image, image, base_masks, masks)
            calculate_and_display_best_matches(base_image, image, base_masks, masks)
        except ValueError as e:
            print(f"Skipping image {idx + 1}: {e}")

def get_mask_by_image_name(image_name, image_files, masks_list):
    """
    Retrieve the mask corresponding to a given image file name.

    :param image_name: The name of the image file to search for.
    :param image_files: List of image file names.
    :param masks_list: List of masks corresponding to the image files.
    :return: The mask corresponding to the given image file name, or None if not found.
    """
    try:
        # Find the index of the image in the list
        index = image_files.index(image_name)
        # Return the corresponding mask
        return masks_list[index]
    except ValueError:
        # If the image file is not found, return None
        print(f"Image file '{image_name}' not found in the list.")
        return None            

def plot_in_grid_with_titles(images_list, names_list, cols=4, figsize=(15, 5)):
    if len(images_list) != len(names_list):
        raise ValueError("The length of images_list and names_list must be the same.")

    num_images = len(images_list)
    num_rows = (num_images + cols - 1) // cols  # Calculate the required number of rows

    fig, axes = plt.subplots(num_rows, cols, figsize=figsize)
    axes = axes.ravel()  # Flatten the axes for easy indexing

    for i in range(len(axes)):
        if i < num_images:
            axes[i].imshow(images_list[i])
            axes[i].set_title(names_list[i], fontsize=10)  # Add title to the image
            axes[i].axis('off')  # Hide axes for better visualization
        else:
            axes[i].axis('off')  # Hide any extra empty subplot spaces

    plt.tight_layout()
    plt.show()

def align_and_plot_images(image_files,base_image_ix, images_list, masks, min_segment_area=0.0015, max_segment_area=1.0):
    """
    Aligns images in the list to the base image using translation, rotation, and homography.
    Displays the base image with bounding boxes of filtered masks, the target image with bounding boxes
    of match results, and the aligned images in a single row for each target image.

    """
    base_image = images_list[base_image_ix]
    # Filter masks based on area thresholds
    filtered_masks = mask_area_filter(base_image, masks, min_segment_area, max_segment_area)

    for target_image_ix, target_image in enumerate(images_list):
        # Match segments between the base image and target image
        match_results = process_all_masks(base_image, filtered_masks, target_image)

        # Align using translation
        translated_image = align_images_using_translation(base_image, target_image, match_results, filtered_masks)

        # Align using translation and rotation
        translated_rotated_image = align_images_using_translation_and_rotation(base_image, target_image, match_results, filtered_masks)

        # Align using homography
        if(len(match_results) > 3):
            homography_aligned_image = align_images_using_homography(base_image, target_image, match_results, filtered_masks)
        else:
            print(f'Only {len(match_results)} matches. Using bboxes in stead of centers')
            homography_aligned_image = align_images_using_homography_corners(base_image, target_image, match_results, filtered_masks)

        # Create copies for drawing bounding boxes
        base_with_masks = base_image.copy()
        target_with_matches = target_image.copy()

        fig, axes = plt.subplots(1, 5, figsize=(25, 5))
        titles = [
            "Base Image with Filtered Masks",
            "Target Image with Matches",
            "Translated",
            "Translated + Rotated",
            "Homography Aligned"
        ]

        # Plot base image with bounding boxes for filtered masks
        axes[0].imshow(base_with_masks)
        axes[0].set_title(image_files[base_image_ix], fontsize=10)
        axes[0].axis('off')

        for idx, mr in enumerate(match_results):
            mask=filtered_masks[mr['mask_index']]
            base_bbox = mask['bbox']
            rect = patches.Rectangle(
                (base_bbox[0], base_bbox[1]),
                base_bbox[2],
                base_bbox[3],
                linewidth=2,
                edgecolor='blue',
                facecolor='none'
            )
            axes[0].add_patch(rect)
            axes[0].text(
                base_bbox[0] + base_bbox[2] / 2,
                base_bbox[1] + base_bbox[3] / 2,
                str(idx),
                color='white',
                fontsize=10,
                ha='center',
                va='center',
                bbox=dict(facecolor='blue', edgecolor='none', alpha=0.5)
            )

        # Plot target image with bounding boxes for match results
        axes[1].imshow(target_with_matches)
        axes[1].set_title(image_files[target_image_ix], fontsize=10)
        axes[1].axis('off')

        for idx, match in enumerate(match_results):
            target_bbox = (
                match['best_match_loc'][0],
                match['best_match_loc'][1],
                match['segment_shape'][1],
                match['segment_shape'][0]
            )
            rect = patches.Rectangle(
                (target_bbox[0], target_bbox[1]),
                target_bbox[2],
                target_bbox[3],
                linewidth=2,
                edgecolor='green',
                facecolor='none'
            )
            axes[1].add_patch(rect)
            axes[1].text(
                target_bbox[0] + target_bbox[2] / 2,
                target_bbox[1] + target_bbox[3] / 2,
                str(idx),
                color='white',
                fontsize=10,
                ha='center',
                va='center',
                bbox=dict(facecolor='green', edgecolor='none', alpha=0.5)
            )

        # Plot the aligned images
        axes[2].imshow(translated_image)
        axes[2].set_title(titles[2], fontsize=10)
        axes[2].axis('off')

        axes[3].imshow(translated_rotated_image)
        axes[3].set_title(titles[3], fontsize=10)
        axes[3].axis('off')

        axes[4].imshow(homography_aligned_image)
        axes[4].set_title(titles[4], fontsize=10)
        axes[4].axis('off')

        plt.tight_layout()
        plt.show()
import re
def save_crop_exif(src_image_path,warped_image, crop_width, crop_height, warped_images_folder):
    rgb_image = cv2.cvtColor(warped_image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_image)    
    if(crop):
        cropped_img = crop_center(pil_image, crop_width, crop_height)
    else:
        cropped_img = pil_image
    opencv_cropped_img = np.array(cropped_img)
    opencv_cropped_img_bgr = opencv_cropped_img
    src_image_path = re.sub(r"_reference", "", src_image_path)
    final_img = add_timestamp_from_exif(f"{src_image_path}", opencv_cropped_img_bgr)
    final_img = Image.fromarray(np.array(final_img))
    final_img.save(f'{warped_images_folder}/{os.path.basename(src_image_path)}')

def convert_pedometer_file(input_file):
    """
    Converts the Pedometer_Backup.txt file into a DataFrame with three columns: date, daysteps, and steps.

    Args:
        input_file (str): Path to the input text file.

    Returns:
        pd.DataFrame: DataFrame with columns date, daysteps, and steps.
    """
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # Skip the first two lines and process the rest
    data = []
    cumulative_steps = 0

    for line in lines[2:]:  # Skip the first two lines
        parts = line.strip().split(',')
        date = parts[0]  # First column is the date

        # Sum columns 2 to 5 (indices 1 to 4 in zero-based indexing)
        daysteps = sum(map(int, parts[1:5]))

        # Calculate cumulative steps
        cumulative_steps += daysteps

        # Append to data
        data.append([date, daysteps, cumulative_steps])

    # Create and return DataFrame
    df = pd.DataFrame(data, columns=["date", "daysteps", "steps"])
    return df


<h1>SAM segmentation</h1>

In [ ]:
# Segment Anything Model initialization
sam2_checkpoint = "./checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

sam2 = build_sam2(model_cfg, sam2_checkpoint, device=device, apply_postprocessing=False)
mask_generator_1 = SAM2AutomaticMaskGenerator(sam2)
mask_generator_2 = SAM2AutomaticMaskGenerator(
    model=sam2,
    points_per_side=32,               # points_per_side: Optional[int] = 32,
    points_per_batch=64,              # points_per_batch: int = 64,
    pred_iou_thresh=0.8,              # pred_iou_thresh: float = 0.8,
    stability_score_thresh=0.95,      # stability_score_thresh: float = 0.95,
    stability_score_offset=1.0,       # stability_score_offset: float = 1.0,
    crop_n_layers=0,                  # crop_n_layers: int = 0,
    box_nms_thresh=0.7,               # box_nms_thresh: float = 0.7,
    crop_n_points_downscale_factor=1, # crop_n_points_downscale_factor: int = 1,
    min_mask_region_area=5.0,        # min_mask_region_area: int = 0,
    use_m2m=False,                     # use_m2m: bool = False,
)
mask_generator=mask_generator_2

image_files,images_list, masks_list = process_images_and_masks(image_path, masks_path, mask_generator_2)
# plot_in_grid_with_titles(images_list,image_files)
# #Load images
# image_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.jpg')])
# # image_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.jpg') and (f==base_image or f=='IMG_20240528_080644.jpg')])
# images_list = [load_image(os.path.join(folder_path, file)) for file in image_files]
# print(f'Loaded {len(images_list)} images from {folder_path}')

# # Segment base image 
# base_image_ix = image_files.index(base_image)
# masks_base_image = mask_generator.generate(images_list[base_image_ix])
# masks_base_image = get_mask_by_image_name(base_image,image_files,masks_list)

In [45]:
# masks_0 = expand_bounding_boxes_in_masks(images_list[base_image_ix], masks_base_image, matched_segments_bbox_expand)
# align_and_plot_images(image_files,base_image_ix,images_list,masks_0, min_segment_area=min_segment_surface,max_segment_area=max_segment_surface)

<h1>Warp images</h1>

In [ ]:
# base_image_ix = image_files.index(base_image)
# masks_0 = expand_bounding_boxes_in_masks(images_list[base_image_ix], masks_base_image, matched_segments_bbox_expand)
matched_results=[]
matched_masks=[]
images_to_warp = images_list[:len(images_list)]

# 1. Match segments
# for idx, image in enumerate(images_to_warp):
#     results, masks = match_segments(images_list[base_image_ix], masks_0, image)
#     matched_results.append(results)
#     matched_masks.append(masks)

# 2. Match and plot matched segments
# for idx, image in enumerate(images_to_warp):
#     results, masks = match_and_plot_segments(image_files[base_image_ix],image_files[idx], images_list[base_image_ix], masks_0, image)
#     matched_results.append(results)
#     # matched_top_masks.append(masks)
#     matched_masks.append(masks)


# 3. Match and warp and plot matched segments and warped image
for idx, image in enumerate(images_to_warp):
    print(f"Working on {image_files[idx]}")
    if(image_files[idx].find('_reference.jpg') != -1):
        base_image_ix = idx
        masks_base_image = get_mask_by_image_name(image_files[base_image_ix],image_files,masks_list)
        masks_0 = expand_bounding_boxes_in_masks(images_list[base_image_ix], masks_base_image, matched_segments_bbox_expand)
        print(f'Using {image_files[base_image_ix]} as reference')
    # results, masks, warped_image = match_warp_and_plot_segments_using_homography(image_files[base_image_ix],image_files[idx],  images_list[base_image_ix], masks_0, image)
    results, masks, warped_image = match_warp_and_plot_segments_using_homography_with_rigid_transform_fallback(image_files[base_image_ix],image_files[idx],  images_list[base_image_ix], masks_0, image)
    
    warped_images_folder = homograpy_warped_images_folder
    
    # results, masks, warped_image = match_warp_and_plot_segments_using_translation(image_files[base_image_ix],image_files[idx],  images_list[base_image_ix], masks_0, image)
    # warped_images_folder = translation_warped_images_folder
    
    # results, masks, warped_image = match_warp_and_plot_segments_using_translation_and_rotation(image_files[base_image_ix],image_files[idx],  images_list[base_image_ix], masks_0, image)
    # warped_images_folder = translation_and_rotation_warped_images_folder
    
    if(warped_image is not None):
        matched_results.append(results)
        matched_masks.append(masks)
        warped_image_bgr = cv2.cvtColor(warped_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(f'{warped_images_folder}/{image_files[idx]}', warped_image_bgr)   
    else:
        print(f"Cant't warp {image_files[idx]}")

<h1>Crop - exif</h1>

In [ ]:
warped_image_files = sorted(os.listdir(homograpy_warped_images_folder))
os.makedirs(cropped_with_exif_images_folder,exist_ok=True)

for file_name in warped_image_files:
    warped_image_file_path = os.path.join(homograpy_warped_images_folder, file_name)
    if file_name.lower().endswith(('.jpg')):
        src_image_file_path = f'{src_image_path}/{file_name}'
        warped_image = cv2.imread(warped_image_file_path)
        if(warped_image is not None):
            save_crop_exif(src_image_file_path,warped_image,crop_width,crop_height,cropped_with_exif_images_folder)
        else:
            print(f"Cant't save_crop_exif {file_name}")

No EXIF datetime for the image
No EXIF datetime for the image


<h1>Stop motion</h1>

In [8]:
import os
import cv2
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
from matplotlib.dates import DateFormatter
from datetime import datetime
import locale

def create_stop_motion_movie_with_steps(input_folder, output_file, csv_file, frame_duration=2, transition_duration=1, fps=30, audio_file=None):
    # Read CSV data
    data = pd.read_csv(csv_file, parse_dates=['date'])

    # Get all jpg files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.jpg')]
    image_files.sort()  # Sort the files to ensure correct order
    print(f"files {image_files}")

    # Get the dimensions of the first image
    first_image = cv2.imread(os.path.join(input_folder, image_files[0]))
    height, width = first_image.shape[:2]

    # Create a temporary video file
    temp_output = 'temp_output.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(temp_output, fourcc, fps, (width, height))

    # Initialize the line chart image
    line_chart_file = 'line_chart.png'
    full_data = data.copy()

    # Loop through each image
    for i in range(len(image_files)):
        current_img = cv2.imread(os.path.join(input_folder, image_files[i]))
        next_img = cv2.imread(os.path.join(input_folder, image_files[(i + 1) % len(image_files)]))
        
        # Resize images to ensure they have the same size
        current_img = cv2.resize(current_img, (width, height))
        next_img = cv2.resize(next_img, (width, height))

        # Extract the date from the filename (format: IMG_yyyymmdd_hhmmss.jpg)
        image_date_str = image_files[i][4:12]  # Extract yyyymmdd
        image_date = datetime.strptime(image_date_str, '%Y%m%d')

        # Filter the CSV data up to the current image date
        filtered_data = data[data['date'] <= image_date]

        # Update the line chart with the filtered data (this keeps the line chart updated with new data)
        create_line_chart(filtered_data, full_data, line_chart_file)

        # Superimpose the line chart on the current image (without blending it)
        current_img_with_chart = overlay_line_chart(current_img, line_chart_file)

        # Hold the current image (with the updated chart)
        for _ in range(fps * frame_duration):
            out.write(current_img_with_chart)

        # Cross-fade the images, but without fading the line chart
        for j in range(int(fps * transition_duration)):
            alpha = j / (fps * transition_duration)
            blended_img = cv2.addWeighted(current_img, 1 - alpha, next_img, alpha, 0)
            # Overlay the line chart on the blended image (to keep it static during the transition)
            blended_img_with_chart = overlay_line_chart(blended_img, line_chart_file)
            out.write(blended_img_with_chart)

    # Hold the last image if audio is longer than the video
    last_img = current_img_with_chart  # The last image with the chart
    video_duration = len(image_files) * (frame_duration + transition_duration)  # In seconds

    # Get the duration of the MP3 audio file
    if audio_file:
        result = subprocess.run(['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', audio_file], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        audio_duration = float(result.stdout)
        
        if audio_duration > video_duration:
            hold_duration = audio_duration - video_duration
            print(f"Holding the last frame for {hold_duration} seconds.")
            # Hold the last image for the remaining duration of the audio
            for _ in range(int(hold_duration * fps)):
                out.write(last_img)

    out.release()

    # FFmpeg command to add the MP3 audio to the video
    ffmpeg_cmd = [
        'ffmpeg',
        '-y',
        '-i', temp_output,            # Input: stop motion movie
        '-i', audio_file,             # Input: MP3 file
        '-c:v', 'libx264',
        '-preset', 'slow',
        '-crf', '23',
        '-vf', f'scale=-2:720',  # Scale to 720p, maintaining aspect ratio
        '-movflags', '+faststart',
        '-c:a', 'aac',
        '-b:a', '128k',
        '-ar', '44100',            # Resample audio to 44.1 kHz
        '-shortest',               # Stops the video when the shorter stream ends (video or audio)
        output_file
    ]

    # Run the FFmpeg command
    subprocess.run(ffmpeg_cmd, check=True)

    # Remove the temporary file
    os.remove(temp_output)

    print(f"Stop motion movie with audio created: {output_file}")

def overlay_line_chart(image, line_chart_file):
    """
    Overlay the line chart in the bottom-right corner of the given image,
    resizing it to 750px wide and 450px high.
    """
    # Load the line chart as an image (with alpha channel for transparency)
    line_chart_img = cv2.imread(line_chart_file, cv2.IMREAD_UNCHANGED)  # Load with transparency

    # Resize the line chart to be 750px wide and 450px high
    line_chart_img_resized = cv2.resize(line_chart_img, (750, 450))

    # Get the dimensions of the resized chart
    chart_height, chart_width = line_chart_img_resized.shape[:2]

    # Define the region of interest (ROI) in the bottom-right corner
    x_offset = image.shape[1] - chart_width - 10  # 10 pixels from the right
    y_offset = image.shape[0] - chart_height - 10  # 10 pixels from the bottom

    # If the line chart has an alpha channel, blend it with the image
    if line_chart_img_resized.shape[2] == 4:
        # Split the line chart into its color channels and alpha channel
        b, g, r, alpha = cv2.split(line_chart_img_resized)

        # Normalize the alpha channel to be between 0 and 1
        alpha = alpha / 255.0

        # Blend the line chart with the image
        for c in range(0, 3):  # Iterate over the B, G, R channels
            image[y_offset:y_offset+chart_height, x_offset:x_offset+chart_width, c] = (
                alpha * line_chart_img_resized[:, :, c] +
                (1 - alpha) * image[y_offset:y_offset+chart_height, x_offset:x_offset+chart_width, c]
            )

    return image

def create_line_chart(filtered_data, full_data, output_file):
    """
    Creates a static line chart from filtered data (up to a specific date) and saves it as an image.
    The chart will have a light grey background with 70% opacity, display steps in 1K units with 1 decimal,
    and annotate the last data point with the date in Dutch format (dd-mon).
    """
    # Set locale to Dutch
    locale.setlocale(locale.LC_TIME, 'nl_NL.UTF-8')  # For Dutch date formatting

    fig, ax = plt.subplots(figsize=(4, 2))  # Create a smaller chart (400x200px)

    # Set light grey background for the figure with 70% opacity
    fig.patch.set_facecolor(mcolors.to_rgba('lightgrey', 0.7))
    ax.set_facecolor(mcolors.to_rgba('lightgrey', 0.7))

    # Plot the filtered data
    ax.plot(filtered_data['date'], filtered_data['steps'], lw=2)

    # Calculate 5% margin for x-axis extension
    date_range = full_data['date'].max() - full_data['date'].min()
    margin = date_range * 0.10  # 5% of the date range

    # Set the x-axis limits with the added margin
    ax.set_xlim(full_data['date'].min(), full_data['date'].max() + margin)

    # Set the y-axis limits
    ax.set_ylim(full_data['steps'].min() * 0.9, full_data['steps'].max() * 1.1)

    # Format y-axis to show steps in 1K units with 1 decimal
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x/1000:.1f}K'))

    # Annotate the last data point if available, also in 1K units
    if not filtered_data.empty:
        last_date = filtered_data['date'].iloc[-1]
        last_value = filtered_data['steps'].iloc[-1] / 1000  # Convert to 1K units
        last_date_str = last_date.strftime('%d %b')  # Format date as 'dd-mon' in Dutch

        ax.annotate(f'{last_value:.1f}K\n{last_date_str}',  # Display value in 1K units with 1 decimal and date
                    xy=(last_date, last_value * 1000),  # Point at the last data point in original scale
                    xytext=(5, 5),  # Slightly offset the text
                    textcoords='offset points',
                    fontsize=10,
                    color='black')

    # Remove axis lines and labels
    ax.set_axis_off()

    # Save the chart as an image with a semi-transparent background
    plt.savefig(output_file, dpi=100)
    plt.close(fig)


In [55]:
input_folder = cropped_with_exif_images_folder
output_file = stop_motion_result_path
create_stop_motion_movie_with_steps(input_folder, output_file, "steps.csv", transition_duration=0.75, audio_file='one_fine_day.mp3')

files ['IMG_20240528_080644.jpg', 'IMG_20240529_073834.jpg', 'IMG_20240603_074710.jpg', 'IMG_20240605_073640.jpg', 'IMG_20240606_075254.jpg', 'IMG_20240607_073713.jpg', 'IMG_20240610_075152.jpg', 'IMG_20240611_074252.jpg', 'IMG_20240612_074106.jpg', 'IMG_20240613_073134.jpg', 'IMG_20240616_104257.jpg', 'IMG_20240617_082205.jpg', 'IMG_20240619_073804.jpg', 'IMG_20240621_083624.jpg', 'IMG_20240626_075046.jpg', 'IMG_20240627_074112.jpg', 'IMG_20240701_074430.jpg', 'IMG_20240703_073302.jpg', 'IMG_20240709_075414.jpg', 'IMG_20240715_080401.jpg', 'IMG_20240717_075449.jpg', 'IMG_20240722_073527.jpg', 'IMG_20240725_074323.jpg', 'IMG_20240726_074016.jpg', 'IMG_20240729_074638.jpg', 'IMG_20240802_073745.jpg', 'IMG_20240819_073730.jpg', 'IMG_20240820_074416.jpg', 'IMG_20240821_074526.jpg', 'IMG_20240823_074138.jpg', 'IMG_20240826_075635.jpg', 'IMG_20240828_080453.jpg', 'IMG_20240830_073307.jpg', 'IMG_20240903_080620.jpg', 'IMG_20240904_080904.jpg', 'IMG_20240906_075809.jpg', 'IMG_20240909_074851.

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

Stop motion movie with audio created: /Users/peter/playground/parkbridge/auto_parkbridge.mp4


[mp4 @ 0x12e614ef0] Starting second pass: moving the moov atom to the beginning of the file
[out#0/mp4 @ 0x12e614e30] video:72355KiB audio:4621KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.389067%
frame= 8803 fps=158 q=-1.0 Lsize=   77276KiB time=00:04:53.36 bitrate=2157.9kbits/s speed=5.27x    
[libx264 @ 0x12e618b00] frame I:36    Avg QP:16.33  size:243420
[libx264 @ 0x12e618b00] frame P:3616  Avg QP:20.35  size: 17901
[libx264 @ 0x12e618b00] frame B:5151  Avg QP:21.66  size:   116
[libx264 @ 0x12e618b00] consecutive B-frames: 20.5%  4.0%  1.7% 73.8%
[libx264 @ 0x12e618b00] mb I  I16..4:  5.0% 54.5% 40.4%
[libx264 @ 0x12e618b00] mb P  I16..4:  2.8%  1.8%  0.1%  P16..4: 40.5%  3.8%  6.5%  0.0%  0.0%    skip:44.5%
[libx264 @ 0x12e618b00] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  3.3%  0.0%  0.0%  direct: 0.1%  skip:96.6%  L0:10.2% L1:88.6% BI: 1.1%
[libx264 @ 0x12e618b00] 8x8 transform intra:41.3% inter:66.4%
[libx264 @ 0x12e618b00] direct mvs  spatial:99.

Show all the masks overlayed on the image.

In [ ]:
for i in images_list:
    plot_overlayed(i,masks_0)
